In [1]:
import cv2

!wget https://chateaut.fr/documents/Bean_clip.avi

--2024-03-06 09:23:43--  https://chateaut.fr/documents/Bean_clip.avi
Resolving chateaut.fr (chateaut.fr)... 213.186.33.24
Connecting to chateaut.fr (chateaut.fr)|213.186.33.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 561913 (549K) [video/x-msvideo]
Saving to: ‘Bean_clip.avi’

Bean_clip.avi       100%[===================>] 548.74K  1.01MB/s    in 0.5s    

2024-03-06 09:23:44 (1.01 MB/s) - ‘Bean_clip.avi’ saved [561913/561913]



In [2]:
# download yolov4 model weights to data folder
# Set up tracker.
tracker_types = ['MIL','KCF']
tracker_type = tracker_types[0] #0 for MIL and 1 for KCF
print('Tracking with: ',tracker_type)

if tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create()
if tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create()

Tracking with:  MIL


In [3]:
# Init video reader and writer
cap = cv2.VideoCapture('Bean_clip.avi')
#
#Initializing a bounding box
#bbox = img
  ## Bean Tracking
#bbox =  (416, 206, 128, 124) # Children
bbox =  (379, 33, 89, 106)   # Bean
#bbox =  (114, 99, 125, 100)  # Women
#
res=(int(cap.get(3)),int(cap.get(4))) #resolution
fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
!rm output.mp4
out = cv2.VideoWriter('output.mp4', fourcc, 10.0, res)
# Init Tracker
status, frame = cap.read()
ok = tracker.init(frame, bbox)
while(1):
    status, frame = cap.read()
    if status==False:
      break
    timer = cv2.getTickCount()
    ok, bbox = tracker.update(frame)
    #print('tracker status: ',ok)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
    if ok:
      p1 = (int(bbox[0]), int(bbox[1]))
      p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
      cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
      # Tracking failure
      cv2.putText(frame, "Tracking failure detected", (100,80),
      cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
      cv2.putText(frame, tracker_type + " Tracker", (100,20),
      cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

    # Display FPS on frame
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50),
    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

    # Display result
    out.write(frame)
out.release()
cap.release()
print('Traking achieved')

rm: cannot remove 'output.mp4': No such file or directory
Traking achieved


In [4]:
# define helper function to display videos
import io
from IPython.display import HTML
from base64 import b64encode
def show_video(file_name, width=640):
  # show resulting deepsort video
  mp4 = open(file_name,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width="{0}" controls>
        <source src="{1}" type="video/mp4">
  </video>
  """.format(width, data_url))

In [5]:
# convert resulting video from avi to mp4 file format
import os
!rm output2.mp4
#path_video = os.path.join("outputs","tracker.avi")
#%cd outputs/
!ffmpeg -y -loglevel panic -i output.mp4 output2.mp4
#%cd ..

# output object tracking video
#path_output = os.path.join("outputs","output.mp4")
#!ls -al
show_video('output2.mp4', width=640)

rm: cannot remove 'output2.mp4': No such file or directory
